In [68]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore
import numpy as np

NUM_GAMES = c.NUM_GAMES_2020_21
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv('../data/years/2020-21/clean/2020-21_Player_PerGame.csv')
t_df = pd.read_csv('../data/years/2020-21/clean/2020-21_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv('../data/years/2020-21/clean/2020-21_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# custom z score function - halve if negative
adjust_z = np.vectorize(lambda x: x if x >= 0 else x / 2)

# ranking algorithm
rating = adjust_z(zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']))
ws = adjust_z(zscore(p_df['WS']))
vorp = adjust_z(zscore(p_df['VORP']))
bpm = adjust_z(zscore(p_df['BPM']))
tmswins = adjust_z(zscore(p_df['TmsWins']))
p_df['PlayerScore'] = 1.5*rating + ws + vorp + bpm + tmswins
p_df = p_df.sort_values('PlayerScore', ascending=False)

p_df.head(50)

,Player,PlayerID,Tms,Pos,Age,G,GS,MP,FG,FGA,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins,PlayerScore
399,Nikola Jokić,jokicni01,DEN,C,25,72,72,34.6,10.2,18.0,...,12.2,3.4,15.6,0.301,9.1,3.0,12.1,8.8,47.0,21.036831
184,Giannis Antetokounmpo,antetgi01,MIL,PF,26,61,61,33.0,10.3,18.0,...,6.9,3.3,10.2,0.244,6.2,2.8,9.0,5.6,46.0,15.119383
470,Stephen Curry,curryst01,GSW,PG,32,63,63,34.2,10.4,21.7,...,6.5,2.5,9.0,0.201,8.3,0.4,8.7,5.8,39.0,13.433252
260,Joel Embiid,embiijo01,PHI,C,26,51,51,31.1,9.0,17.6,...,5.6,3.2,8.8,0.266,6.3,1.2,7.5,3.8,49.0,12.480924
95,Damian Lillard,lillada01,POR,PG,30,67,67,35.8,9.0,19.9,...,9.6,0.8,10.4,0.209,7.5,-1.3,6.3,5.0,42.0,12.442502
339,Luka Dončić,doncilu01,DAL,PG,21,66,66,34.3,9.8,20.5,...,5.1,2.6,7.7,0.163,6.1,0.7,6.8,5.1,42.0,12.207213
293,Kawhi Leonard,leonaka01,LAC,SF,29,52,52,34.1,8.9,17.5,...,6.4,2.4,8.8,0.238,6.0,1.3,7.3,4.2,47.0,11.566610
257,Jimmy Butler,butleji01,MIA,SF,31,52,52,33.6,7.0,14.2,...,6.6,2.7,9.3,0.255,5.4,2.3,7.7,4.3,40.0,11.121158
450,Rudy Gobert,goberru01,UTA,C,28,71,71,30.8,5.5,8.2,...,6.1,5.2,11.3,0.248,2.1,2.8,4.9,3.8,52.0,11.055179
333,LeBron James,jamesle01,LAL,PG,36,45,45,33.4,9.4,18.3,...,3.0,2.6,5.6,0.179,5.9,2.3,8.1,3.8,42.0,10.230480
